In [1]:
!pip install langdetect #install necessary libraries
!pip install pathlib
!pip install iso639

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=1cf5acc43863dd09f51f30ad4cc0d132ca409192bfeeca36fcbbb42dfcd26723
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect
  Preparing metadata (setup.py) ... done
  Created wheel for iso639: filename=iso639-0.1.4-py3-none-any.whl size=11171 sha256=1923527520b9ae7e1f36385785e4131d670af9f7e279daf277c09574d3571425
  Stored in directory: /root/.cache/pip/wheels/44/48/39/29e97705b560a2fd4dc0f1e272f0f9b36ef403d8a66d431a69
Successfully built iso639


In [2]:
#import necessary libraries
from google import genai
from google.genai import types
import wave
from google.colab import files
import os
from langdetect import detect, DetectorFactory
import iso639
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

In [3]:
client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))

In [4]:
# Set up the wave file to save the output:
def wave_file(filename, pcm, channels=1, rate=24000, sample_width=2):
   with wave.open(filename, "wb") as wf:
      wf.setnchannels(channels)
      wf.setsampwidth(sample_width)
      wf.setframerate(rate)
      wf.writeframes(pcm)


In [5]:
'''This function, when called within one of translation functions, prompts you to search for a file on your computer to upload
  and creates a specific path that can be utlized by the client.file.upload() function
'''

def upload_file():
  uploaded_file=files.upload() #stores file as a dictionary where the key is the filename, and the value is the file content in bytes

  if not uploaded_file: #if the file selection was canceled or no file is chosen, the program will terminate.
    print("No file uploaded. Exiting.")
    exit()
  else:
    uploaded_filename=list(uploaded_file.keys())[0] #extracts the filename from the uploaded_file dictionary
    file_content_bytes=uploaded_file[uploaded_filename] #retrieves the content of the uploaded file using the extracted filename as the key in the dictionary
    local_file_path=uploaded_filename #assigns filename to variable

    with open(local_file_path,'wb') as f:
      f.write(file_content_bytes)
    return local_file_path #opens a new file with the same name as the local_file path and writes in the content stored into the new file.


In [6]:
def t3translate(lang):
  user_input=input("Prompt:\n")

  transcript = client.models.generate_content(
  model="gemini-2.0-flash",
  contents=[f"In one line, translate the following text into {lang}",user_input]).text

  detected_language=iso639.Lang(detect(user_input)) #Uses langdetect library to detect the language that the prompt was submitted in, and uses iso639 to convert the abbreviated language name into the full name (en -> English)
  print(f"Detected Language: {detected_language}\nTranslation: {transcript}")

In [7]:
def t2s_translate(lang):
  user_input=input("Prompt:\n")
  transcript = client.models.generate_content(
  model="gemini-2.0-flash",
  contents=[f"Repeat what was said as input in this language: {lang}",user_input]).text
  detected_language=iso639.Lang(detect(user_input))

  #model and voice selection
  response = client.models.generate_content(
    model="gemini-2.5-flash-preview-tts",
    contents=transcript, # Text to be spoken
    config=types.GenerateContentConfig(
        response_modalities=["AUDIO"],
        speech_config=types.SpeechConfig(
          voice_config=types.VoiceConfig(
              prebuilt_voice_config=types.PrebuiltVoiceConfig(
                voice_name='Gacrux',
              )
          )
        ),
    )
  )

  data = response.candidates[0].content.parts[0].inline_data.data
  print(f"Detected Language: {detected_language}")
  file_name='out.wav' # file name
  wave_file(file_name, data) # Saves the file to current directory
  print("Translated file created!")

In [9]:
def s2t_translate(lang):
  print("Add an audio file.")
  myfile=client.files.upload(file=upload_file())
  transcript = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[f"Repeat what was said in this file in {lang}.",myfile]).text
  transcript2=client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[f"Can you give me a transcript of my file?",myfile]).text
  detected_language=iso639.Lang(detect(transcript2))

  print(f"Detected Language:{detected_language}\n Translation:{transcript}")


In [10]:
def s2s_translate(lang):
  print("Add an audio file.")
  myfile=client.files.upload(file=upload_file())
  transcript = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[f"Repeat what was said in this file in {lang}.",myfile]).text
  transcript2=client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[f"Can you give me a transcript of my audio file",myfile]).text
  detected_language=iso639.Lang(detect(transcript2))

  response = client.models.generate_content(
    model="gemini-2.5-flash-preview-tts",
    contents=transcript, # Text to be spoken
    config=types.GenerateContentConfig(
        response_modalities=["AUDIO"],
        speech_config=types.SpeechConfig(
          voice_config=types.VoiceConfig(
              prebuilt_voice_config=types.PrebuiltVoiceConfig(
                voice_name='Gacrux',
              )
          )
        ),
    )
  )
  data = response.candidates[0].content.parts[0].inline_data.data
  print(f"Detected Language: {detected_language}")
  file_name='out.wav' # file name
  wave_file(file_name, data) # Saves the file to current directory


In [7]:
t3translate("French")

Prompt:
I want food


ServerError: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'Visibility check was unavailable. Please retry the request and contact support if the problem persists', 'status': 'UNAVAILABLE'}}

In [12]:
t2s_translate("Haitian Creole")

Prompt:
I am a student at Pace University.


ServerError: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'Visibility check was unavailable. Please retry the request and contact support if the problem persists', 'status': 'UNAVAILABLE'}}

In [13]:
s2t_translate("German")

Add an audio file.


Saving Test1.wav to Test1.wav


ServerError: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'Visibility check was unavailable. Please retry the request and contact support if the problem persists', 'status': 'UNAVAILABLE'}}

In [ ]:
s2s_translate("French")

Add an audio file.


Saving Test1.wav to Test1.wav
File saved to local path: Test1.wav
Detected Language: fr


In [ ]:
feature_input=input("Which translation would you like to use?: Text-to-text, Text-to-speech, Speech-to-text, or Speech-to-speech? (ttt,tts,stt,sts)")
lang_input=input("What language would you like to have your output translated to?")
if feature_input=="ttt":
  t3translate(lang_input)
elif feature_input=="tts":
  t2s_translate(lang_input)
elif feature_input=="sts":
  s2s_translate(lang_input)
elif feature_input=="stt":
  s2t_translate(lang_input)